# Определение текущего уровеня потребительской лояльности (NPS) телекоммуникационной компании среди клиентов из России. 

**Описание проекта**  

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.  
Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».  
Компания провела опрос и попросила подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.  
Чтобы оценить результаты опроса, оценки обычно делят на три группы:  
- 9-10 баллов — «cторонники» (англ. promoters);  
- 7-8 баллов — «нейтралы» (англ. passives);  
- 0-6 баллов — «критики» (англ. detractors).  
Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».  

**Цель проекта**  

1. Построить дашборд, который представит информацию о текущем уровне NPS среди клиентов и покажет, как этот уровень меняется в зависимости от пользовательских признаков.  
2. ответить на вопросы:  
- Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?  
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?  
- Какой общий NPS среди всех опрошенных?  
- Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?  
3. Ответы на вопросы оформить в виде презентации

In [1]:
# импортируем необходимые библиотеки
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

Предварительно проанализировав данные, будем делать SQL запрос со следующими условиями:
- тип данных в столбце `age` - float - поменяем на int
- разделим клиентов на новые и старые (`new client` / `old client`)
- поменяем поле gender_segment вместо 0 и 1 на `man` и `woman`
- поменяем регистр и разобъем на категории столбцы с типом операционной системы и типом устройства
- корректно пропишем города, где это необходимо
- возмем только необходимую часть из столбцов с сегментами (возростной, по объёму потребляемого трафика и по количеству дней «жизни»)
- результаты опроса, разделим на три группы
- в столбцах `age` и `gender_segment` были пропуски, в запросе исключим их  

После предобработки, в наших данных осталось 501152 строк.  
Удалили 0,27% строк, что в данном случае допустимо и не повлияет на далнейший ход исследования.

In [3]:
query = """
SELECT u.user_id,
       u.lt_day,
       CASE 
           WHEN u.lt_day <= 365 THEN 'new client' 
           ELSE 'old client' 
           END as is_new,
       CAST(u.age AS INT) as age,
       CASE 
           WHEN u.gender_segment = 1 THEN 'woman' 
           ELSE 'man' 
           END as gender_segment,
       CASE
           WHEN u.os_name = 'ANDROID' THEN 'Android'
           WHEN u.os_name = 'PROPRIETARY' THEN 'Proprietary'
           WHEN u.os_name = 'OTHER' OR os_name = 'unknown' THEN 'Other'
           WHEN u.os_name IN ('WINDOWS PHONE', 'WINDOWS MOBILE') THEN 'Windows'
           WHEN u.os_name = 'SYMBIAN OS' THEN 'Symbian OS'
           WHEN u.os_name = 'BADA OS' THEN 'Bada OS'
           ELSE u.os_name END as os_name,
       CASE
           WHEN u.cpe_type_name = 'SMARTPHONE' THEN 'Smartphone'
           WHEN u.cpe_type_name = 'TABLET' THEN 'Tablet'
           WHEN u.cpe_type_name IN ('PHONE', 'MOBILE PHONE/FEATURE PHONE') THEN 'Phone'
           WHEN u.cpe_type_name IN ('ROUTER', 'WLAN ROUTER', 'MIFI ROUTER') THEN 'Router'
           WHEN u.cpe_type_name IN ('USB MODEM', 'MODEM') THEN 'Modem'
           ELSE 'Other' END as cpe_type_name,
       l.country,
       CASE
           WHEN l.city = 'СанктПетербург' THEN 'Санкт-Петербург'
           WHEN l.city = 'НижнийНовгород' THEN 'Нижний Новгород'
           WHEN l.city = 'РостовнаДону' THEN 'Ростов-на-Дону'
           WHEN l.city = 'НабережныеЧелны' THEN 'Набережные Челны'
           WHEN l.city = 'УланУдэ' THEN 'Улан-Удэ'
           WHEN l.city = 'НижнийТагил' THEN 'Нижний Тагил'
           ELSE l.city END as city,
       SUBSTRING(ags.title, 4) as age_segment,
       SUBSTRING(ts.title, 4) as traffic_segment,
       SUBSTRING(ls.title,4) as lifetime_segment,
       u.nps_score,
       CASE 
           WHEN u.nps_score >= 9 THEN 'promoters'
           WHEN u.nps_score >= 7 THEN 'passives'
           ELSE 'detractors'
           END as nps_group
FROM user as u
JOIN location as l ON u.location_id = l.location_id
JOIN age_segment as ags ON u.age_gr_id=ags.age_gr_id
JOIN traffic_segment ts ON u.tr_gr_id=ts.tr_gr_id
JOIN lifetime_segment ls ON u.lt_gr_id=ls.lt_gr_id
WHERE u.age AND u.gender_segment IS NOT NULL; 
"""

In [4]:
df = pd.read_sql(query, engine)
#df.sample(10)

In [5]:
#df.info()

In [6]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

In [7]:
df

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old client,45,woman,Android,Smartphone,Россия,Уфа,45-54,1-5,36+,10,promoters
1,A001WF,2344,old client,53,man,Android,Smartphone,Россия,Киров,45-54,1-5,36+,10,promoters
2,A003Q7,467,old client,57,man,Android,Smartphone,Россия,Москва,55-64,20-25,13-24,10,promoters
3,A004TB,4190,old client,44,woman,IOS,Smartphone,Россия,Ростов-на-Дону,35-44,0.1-1,36+,10,promoters
4,A004XT,1163,old client,24,man,Android,Smartphone,Россия,Рязань,16-24,5-10,36+,10,promoters
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501147,ZZZKLD,1249,old client,54,woman,Android,Smartphone,Россия,Москва,45-54,1-5,36+,5,detractors
501148,ZZZLWY,129,new client,31,man,Android,Smartphone,Россия,Москва,25-34,1-5,4-6,8,passives
501149,ZZZQ5F,522,old client,36,man,Android,Smartphone,Россия,Сургут,35-44,25-30,13-24,10,promoters
501150,ZZZQ8E,2936,old client,37,woman,Android,Smartphone,Россия,Улан-Удэ,35-44,65-70,36+,9,promoters


Ссылка на tableau pablic


https://public.tableau.com/views/NPStelecomcompany/PRESENTATION?:language=en-US&:display_count=n&:origin=viz_share_link